In [2]:
from google.colab import drive
drive.mount('/content/drive')
# drive.mount('/content/drive', force_remount=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
! ls /content/drive/MyDrive/ubc_data_relevant/
ubc_data_path = '/content/drive/MyDrive/ubc_data_relevant_splitted'
ubc_input_path = f'{ubc_data_path}/input'
target_path = f'{ubc_data_path}/target'

add_to_cart.parquet  product_buy.parquet	 search_query.parquet
input		     product_properties.parquet  target
page_visit.parquet   remove_from_cart.parquet


In [49]:
! rm -rf recsys2025_challenge

In [50]:
! test -f recsys2025_challenge || git clone --recurse-submodules --remote-submodules https://github.com/zmrocze/recsys2025_challenge.git

Cloning into 'recsys2025_challenge'...
remote: Enumerating objects: 311, done.
remote: Counting objects: 100% (311/311), done.
remote: Compressing objects: 100% (185/185), done.
remote: Total 311 (delta 210), reused 224 (delta 123), pack-reused 0 (from 0)
Receiving objects: 100% (311/311), 2.22 MiB | 28.37 MiB/s, done.
Resolving deltas: 100% (210/210), done.
Submodule 'recsys2025' (https://github.com/Synerise/recsys2025.git) registered for path 'recsys2025'
Cloning into '/content/recsys2025_challenge/recsys2025'...
remote: Enumerating objects: 71, done.        
remote: Counting objects: 100% (19/19), done.        
remote: Compressing objects: 100% (10/10), done.        
remote: Total 71 (delta 9), reused 9 (delta 9), pack-reused 52 (from 1)        
Receiving objects: 100% (71/71), 47.89 KiB | 15.96 MiB/s, done.
Resolving deltas: 100% (20/20), done.
Submodule path 'recsys2025': checked out '25ec48089a8e3516928b9a402dea4c8dba6753d2'


In [6]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import os
import sys

# Below paths: bad but managable

module_path = os.path.abspath(os.path.join('./recsys2025_challenge'))
sys.path.append(module_path)
module_path = os.path.abspath(os.path.join('./recsys2025_challenge/recsys2025'))
sys.path.append(module_path)
module_path = os.path.abspath(os.path.join('./recsys2025_challenge/src/our_lib'))
# print(module_path)
sys.path.append(module_path)

# sys.path.insert(1, "./recsys2025_challenge/src/our_lib")

# from our_lib import *
# from src.our_lib import RecSysData
# from our_lib import split_data, create_baseline_embeddings, contest_training
# from our_lib import validate_and_load_embeddings

In [7]:
! pip install pytorch-lightning torchmetrics neptune pyarrow
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

2.6.0+cu124
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


# load data

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [9]:
# load everything
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
import torchmetrics as tm

In [32]:
from src.our_lib import RecSysData, unique_edges, load_target, drop_duplicates, all_categories_in_df_numpy, all_users_in_df_numpy, all_items_in_df_numpy, join_item_with_category

In [10]:
input_data = RecSysData.read_parquet(ubc_input_path)
product_properties = pd.read_parquet(f'{ubc_data_path}/product_properties.parquet')
all_input_users = input_data.all_users_set()
all_categories = all_categories_in_df_numpy(product_properties)
input_data

RecSysData(
  add_to_cart=1922253,
  page_visit=32941261,
  product_buy=939835,
  product_properties=1197634,
  remove_from_cart=800359,
  search_query=3204721,
  all users: 858489,
  all products: 674374
  all urls: 5185400,
)

In [11]:
delattr(input_data, 'page_visit')
delattr(input_data, 'search_query')

In [12]:
target_data = load_target(target_path)
train_target = target_data['train_target']
valid_target = target_data['validation_target']

In [13]:
input_data.add_to_cart = join_item_with_category(input_data.add_to_cart, product_properties)
input_data.remove_from_cart = join_item_with_category(input_data.remove_from_cart, product_properties)
input_data.product_buy = join_item_with_category(input_data.product_buy, product_properties)

In [14]:
first_train_timestamp = train_target['timestamp'].min()
first_valid_timestamp = valid_target['timestamp'].min()

print(f"First timestamp in train_target: {first_train_timestamp}")
print(f"First timestamp in valid_target: {first_valid_timestamp}")

First timestamp in train_target: 2022-09-13 00:03:35
First timestamp in valid_target: 2022-09-27 00:00:15


In [15]:
print(input_data.add_to_cart['timestamp'].min())
print(input_data.remove_from_cart['timestamp'].min())
print(input_data.product_buy['timestamp'].min())
print(input_data.add_to_cart['timestamp'].max())
print(input_data.remove_from_cart['timestamp'].max())
print(input_data.product_buy['timestamp'].max())

2022-05-23 00:10:15
2022-05-23 00:12:20
2022-05-23 00:10:40
2022-09-12 23:59:45
2022-09-12 23:58:10
2022-09-12 23:58:35


In [16]:
# drop duplicates
aaxxrta = 'category' # 'sku'
drop_duplicates(input_data.add_to_cart, subset=['client_id', aaxxrta])
drop_duplicates(input_data.remove_from_cart, subset=['client_id', aaxxrta])
drop_duplicates(input_data.product_buy, subset=['client_id', aaxxrta])

drop_duplicates(train_target, subset=['client_id', aaxxrta])
drop_duplicates(valid_target, subset=['client_id', aaxxrta])

# print all the lengths
print(f"add_to_cart: {len(input_data.add_to_cart)}")
print(f"remove_from_cart: {len(input_data.remove_from_cart)}")
print(f"product_buy: {len(input_data.product_buy)}")
print(f"train_target: {len(train_target)}")
print(f"valid_target: {len(valid_target)}")

print(f"len(users) x len(categories): {len(all_input_users) * len(all_categories)}")

add_to_cart: 1922253
remove_from_cart: 800359
product_buy: 939835
train_target: 130109
valid_target: 148479
len(users) x len(categories): 5480593776


In [17]:
# input_data.print_datasets_stats_nicely("input data")

In [18]:
train_users_set = set(train_target['client_id'].unique())
valid_users_set = set(valid_target['client_id'].unique())

In [19]:
# Get users who have only one interaction across all input data types
all_interactions = pd.concat([
  input_data.add_to_cart[['client_id']],
  input_data.remove_from_cart[['client_id']],
  input_data.product_buy[['client_id']]
])

user_interaction_counts = all_interactions['client_id'].value_counts()
single_interaction_users = set(user_interaction_counts[user_interaction_counts == 1].index)
users_not_in_test = single_interaction_users.difference(valid_users_set).difference(train_users_set)

print(f"Users with single interaction in input_data: {len(single_interaction_users)}")
print(f"Users with single interaction and no interaction in test set: {len(users_not_in_test)}")

Users with single interaction in input_data: 190183
Users with single interaction and no interaction in test set: 181914


In [20]:
# removing these, no information gained
input_data.add_to_cart = input_data.add_to_cart[~input_data.add_to_cart['client_id'].isin(users_not_in_test)]
input_data.remove_from_cart = input_data.remove_from_cart[~input_data.remove_from_cart['client_id'].isin(users_not_in_test)]
input_data.product_buy = input_data.product_buy[~input_data.product_buy['client_id'].isin(users_not_in_test)]

In [21]:
# sanity check
_known_categories = set(input_data.add_to_cart['category'].unique()).union(
    set(input_data.remove_from_cart['category'].unique())).union(
    set(input_data.product_buy['category'].unique()))
known_users = set(input_data.add_to_cart['client_id'].unique()).union(
    set(input_data.remove_from_cart['client_id'].unique())).union(
    set(input_data.product_buy['client_id'].unique()))

len(_known_categories), len(known_users), len(all_categories)

(6130, 485297, 6384)

In [22]:
train_target_known = train_target[train_target['client_id'].isin(known_users)]
valid_target_known = valid_target[valid_target['client_id'].isin(known_users)]
print(f"train_target_known: {len(train_target_known)}")
print(f"valid_target_known: {len(valid_target_known)}")

train_target_known: 60108
valid_target_known: 63112


# training gat

In [51]:
# imports
# from src.our_lib import NodeIdMap, JustGAT, RecGAT, DotproductEdgePredictor, LinearEdgePredictor, BprLossLoader
# from src.our_lib import test_out, l2_reg, train_test_split_pos_edges, int_tensor, unique_edges
from src.our_lib import create_val_edge_batched, NodeIdMap, JustGAT, RecGAT, DotproductEdgePredictor, LinearEdgePredictor, BprLossLoader, test_out, l2_reg, train_test_split_pos_edges, int_tensor, unique_edges, loss_f, BprTraining, RecLightGCN, JustLightGCN
import random

In [24]:
### NOTE!:
### There are 2 types of ids: product has sku    and the id of a product node in graph (can exchange with node_id_map.item_of_id/id_of_item).
#                             user has client_id and the id of a user    node in graph

node_id_map = NodeIdMap(known_users, all_categories)
train_edge_index = node_id_map.make_edges(train_target_known.client_id.values, train_target_known.category.values).to(device=device)
val_edge_index = node_id_map.make_edges(valid_target_known.client_id.values, valid_target_known.category.values).to(device=device)

N = node_id_map.N

In [25]:
# TODO: take negatives from batch items, not from all
neg_samples = 5
batch_size = 1024
### these are edges used for loss calculation
### and they define the graph (maybe together with product_buy, remove_from_cart)
train_loader = BprLossLoader(
    edge_index=train_edge_index,
    trg_index_range=(node_id_map.n_users, node_id_map.N), # node index range for all items (used to sample negative items)
    batch_size=batch_size,
    neg_samples=neg_samples,
    random_state=42,
    device=device
)

test_loader = BprLossLoader(
  edge_index=val_edge_index,
  trg_index_range=(node_id_map.n_users, node_id_map.N),  # node index range for all items (used to sample negative items)
  batch_size=batch_size,
  neg_samples=neg_samples,
  random_state=42,
  device=device
)

In [27]:
auroc_batch_size=100
val_edge_index_batched = create_val_edge_batched(node_id_map, val_edge_index, auroc_batch_size, device='cpu')

In [52]:
embedding_dim = 64
dropout_gat=0.3
edge_dim = 3
num_layers=2
init_type='normal'
init_a=0.001
dropout_edge=0.5
lr=0.001
l2_reg=0.0001
accumulate_grad_batches=1
check_val_every_n_epoch=5
patience=5
lr_reduce_factor=0.5
lr_scheduler_monitor="train_loss"
max_epochs=50

def make_model():
  ### define graph
  # new_recgat = lambda: RecGAT(users, items, embedding_dim=embedding_dim, edge_dim=edge_dim, num_layers=2, device=device, dropout=0.3, type='uniform', a=0.0000001)
  # new_recgat = lambda: RecGAT(known_users, all_categories, embedding_dim=embedding_dim, edge_dim=edge_dim, num_layers=num_layers, device=device, dropout=dropout_gat, type=init_type, a=init_a)
  new_gcn = lambda: RecLightGCN(known_users, all_categories, embedding_dim=embedding_dim, num_layers=num_layers, device=device, type=init_type, a=init_a)
  # recgat = new_recgat()
  recgat = new_gcn()

  # edge_attr_add_to_cart = torch.tensor([[1, 0, 0]], dtype=torch.float).repeat(len(input_data.add_to_cart), 1).to(device)
  # edge_attr_remove_from_cart = torch.tensor([[0, 1, 0]], dtype=torch.float).repeat(len(input_data.remove_from_cart), 1).to(device)
  # edge_attr_product_buy = torch.tensor([[0, 0, 1]], dtype=torch.float).repeat(len(input_data.product_buy), 1).to(device)

  # recgat.add_edges_from_user_category_df(input_data.add_to_cart, edge_attr=edge_attr_add_to_cart)
  # recgat.add_edges_from_user_category_df(input_data.remove_from_cart, edge_attr=edge_attr_remove_from_cart)
  # recgat.add_edges_from_user_category_df(input_data.product_buy, edge_attr=edge_attr_product_buy)
  recgat.add_edges_from_user_category_df(input_data.add_to_cart)
  # recgat.add_edges_from_user_category_df(input_data.remove_from_cart)
  recgat.add_edges_from_user_category_df(input_data.product_buy)

  ### edge predictor (here: only for add_to_cart edges. todo: predict more stuff (like next purchased category in next 14days from propensity categories) with more varied predictors and combine losses)
  edge_predictor = LinearEdgePredictor(embedding_dim=embedding_dim, dropout=dropout_edge).to(device=device)

  model = BprTraining(
      recgat=recgat,
      edge_predictor=edge_predictor,
      lr=lr,
      l2_reg=l2_reg,
      # propensity_sku=sampled_items, # for auroc calc
      auroc_batch_size=auroc_batch_size,
      retain_grad=(accumulate_grad_batches>1),
      val_edge_index_batched=val_edge_index_batched,
      val_edge_index=val_edge_index,
      patience=patience,
      factor=lr_reduce_factor,
      lr_scheduler_monitor=lr_scheduler_monitor,
      device=device,
      # forward_gat_every_n = 20, ### tradeoff : speed vs training stability
      # this has to be fixed and work with n>1, this is slow, use big batch to make up for it:
      # forward_gat_every_n = accumulate_grad_batches,
    )

  ### sillynes
  # model = remake_model_if_loss_nan(model)
  return model


In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import WandbLogger

model_checkpoint_val = ModelCheckpoint(
    monitor='val_loss',
    auto_insert_metric_name=True,
    save_top_k=2,
    mode='min',
)
model_checkpoint_auroc = ModelCheckpoint(
  monitor='val_auroc',
  auto_insert_metric_name=True,
  mode = 'max',
  save_top_k=2,
)

# lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau()

# model = remake_model_if_loss_nan(make_model)
model = make_model()

log_model = True
trial_k = random.randint(0, 100000)
print(trial_k)
wandb_logger = WandbLogger(
  # project="recsys_categories",
  project="recsys2025",
  name=f"recgat_bpr_light_{trial_k}",
  log_model=True
  )
wandb_logger.experiment.config.update({
  "embedding_dim": embedding_dim,
  "edge_dim": edge_dim,
  "num_layers": num_layers,
  "dropout_gat": dropout_gat,
  "init_type": init_type,
  "init_a": init_a,
  "dropout_edge": dropout_edge,
  "lr": lr,
  "l2_reg": l2_reg,
  "batch_size": batch_size,
  "auroc_batch_size": auroc_batch_size,
  "accumulate_grad_batches": accumulate_grad_batches,
  "neg_samples": neg_samples,
  "log_model": log_model,
  "graph_items": model.recgat.node_id_map.n_items,
  "graph_users": model.recgat.node_id_map.n_users,
  "graph_edges_n": model.recgat.edge_index.size(1)
})

trainer = pl.Trainer(  max_epochs=max_epochs
                     , check_val_every_n_epoch=check_val_every_n_epoch
                     , logger=wandb_logger
                    #  , callbacks=[model_checkpoint_auroc, model_checkpoint_val, ]
                     , accumulate_grad_batches=accumulate_grad_batches
                      #,   retain_grad=True
                    #  , terminate_on_nan=True
                     )

# print("test_loader", test_loader)
# print("test_edges.shape", test_edges.shape)
wandb_logger.watch(model, log="all")

trainer.fit(model, train_loader, val_dataloaders=test_loader)

INFO:pytorch_lightning.utilities.rank_zero:Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name           | Type                | Params | Mode 
---------------------------------------------------------------
0 | recgat         | RecLightGCN         | 31.5 M | train
1 | edge_predictor | LinearEdgePredictor | 8.3 K  | train
---------------------------------------------------------------
31.5 M    Trainable params
0         Non-trainable pa

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]